In [26]:
# Load dataset

import pandas as pd

data_xls = pd.read_excel('Training data.xlsx', 'Info', index_col=None)
data_xls.to_csv('Info.csv', encoding='utf-8',index = False)
data_xls = pd.read_excel('Training data.xlsx', 'TPR', index_col=None)
data_xls.to_csv('TPR.csv', encoding='utf-8', index = False)

infoSet = pd.read_csv("Info.csv", header = None, skiprows=[0])
TPRSet = pd.read_csv("TPR.csv", header = None, skiprows=[0])

TPRSet

,0,1,2,3,4,5,6
0,12584,2019-09-27 13:10:19,38.3,96.0,22.0,137.0,91.0
1,12584,2019-09-27 13:30:21,38.8,NaN,NaN,NaN,NaN
2,12584,2019-09-27 15:03:55,39.9,NaN,NaN,NaN,NaN
3,12584,2019-09-27 16:30:46,38.5,71.0,20.0,104.0,41.0
4,12584,2019-09-27 17:36:51,NaN,77.0,25.0,111.0,44.0
...,...,...,...,...,...,...,...
8746,98043021,2018-08-28 09:00:28,36.5,76.0,18.0,116.0,70.0
8747,98043021,2018-08-28 12:25:18,NaN,86.0,NaN,126.0,80.0
8748,98043021,2018-08-28 14:30:53,37.1,93.0,17.0,148.0,95.0
8749,98043021,2018-08-28 19:15:01,37.0,85.0,18.0,129.0,73.0


In [27]:
# Merge info set and TPR set
from sklearn.decomposition import PCA

aggregation_functions = {2: 'mean', 3: 'mean', 4: 'mean', 5: 'mean', 6: 'mean'}

TPRaggreg = TPRSet.groupby(TPRSet[0]).aggregate(aggregation_functions)
mergeSet = pd.merge(infoSet.iloc[:, [True, True, True, False, False, False, False]], TPRaggreg.iloc[:, [True, True, True, True, True]], on=0)
mergeSet

,0,1,2_x,2_y,3,4,5,6
0,12584,1,44,37.818421,87.764706,21.294118,113.055556,67.823529
1,82244,2,75,37.000000,84.684211,19.750000,131.789474,70.684211
2,90395,2,92,37.012500,66.777778,18.312500,141.444444,65.166667
3,113952,2,42,37.036364,75.866667,17.533333,130.882353,86.529412
4,146267,1,41,37.219048,82.352941,16.705882,111.250000,70.125000
...,...,...,...,...,...,...,...,...
315,89837721,2,75,37.419231,99.687500,19.562500,126.000000,75.000000
316,90073821,2,40,37.597143,90.576923,17.888889,142.842105,72.368421
317,95012241,2,64,37.847727,91.411765,19.266667,138.687500,74.500000
318,97192721,1,57,37.657143,103.185185,18.000000,172.444444,107.851852


In [28]:
# Extract feature list and clean strings
import numpy as np

def str_trim(series):
    series = series.replace(' ', np.nan)
    series = series.replace('', np.nan, regex=True)
    series = series.replace("0", np.nan, regex=True)
    
    feature = pd.DataFrame(series.str.split(',').explode())
    feature = feature.drop_duplicates(subset=None, keep='first', inplace=False)
    feature = feature.replace('', np.nan)
    feature = feature.dropna()

    return series, feature

bacteria = infoSet.iloc[:, 5]
bacteria = bacteria.rename("bacteria")
bacteriaSet, bacteria_feature= str_trim(bacteria)
bacteria_feature = bacteria_feature.to_numpy().flatten()

common = infoSet.iloc[:, 3]
common = common.rename("common")
commonSet, commonFeature = str_trim(common)
commonFeature = commonFeature.to_numpy().flatten()
# mergeSet[5]

In [29]:
# Create one hot encoding
def extract_df(ser, features):
    df= pd.DataFrame()
    for feature in features:
        id = 0
        new_list = []
        for element in ser:
            if feature == element:
                new_list.append(1)
            else:
                new_list.append(0)
    
        df[feature] = new_list
    return df
            
#origin_ant
bacteria_df = extract_df(bacteriaSet, bacteria_feature)
common_df = extract_df(commonSet, commonFeature)
common_df

,1,2,4
0,1,0,0
1,1,0,0
2,0,0,0
3,0,0,0
4,0,1,0
...,...,...,...
315,0,0,0
316,1,0,0
317,0,0,0
318,0,0,1


In [30]:
# Create cleaned data set
import numpy as np
from sklearn import preprocessing


# 創造 dummy variables
label_encoder = preprocessing.LabelEncoder()
encoded_no = label_encoder.fit_transform(mergeSet[0])

# 建立訓練與測試資料
patient_x = pd.concat([mergeSet, bacteria_df, common_df], axis=1)
patient_x = patient_x.iloc[:, 1:]
patient_y = infoSet.iloc[:, 6]

patient_x
# type(patient_x)

,1,2_x,2_y,3,4,5,6,Escherichia coli,Yeast-like,Staphylococcus aureus (MRSA),...,Acinetobacter baumannii,Staphylococcus aureus,Candida tropicalis,Enterobacter cloacae ssp cloacae,Candida glabrata,Corynebacterium striatum,Staphylococcus haemolyticus,1,2,4
0,1,44,37.818421,87.764706,21.294118,113.055556,67.823529,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,75,37.000000,84.684211,19.750000,131.789474,70.684211,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,92,37.012500,66.777778,18.312500,141.444444,65.166667,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,42,37.036364,75.866667,17.533333,130.882353,86.529412,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,41,37.219048,82.352941,16.705882,111.250000,70.125000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,2,75,37.419231,99.687500,19.562500,126.000000,75.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
316,2,40,37.597143,90.576923,17.888889,142.842105,72.368421,0,0,0,...,0,0,0,0,1,0,0,1,0,0
317,2,64,37.847727,91.411765,19.266667,138.687500,74.500000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
318,1,57,37.657143,103.185185,18.000000,172.444444,107.851852,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
# Reduce demensionality
from sklearn.feature_selection import SelectKBest, f_classif, chi2, SelectFpr, mutual_info_classif


kb = SelectKBest(mutual_info_classif, k=10).fit(patient_x, patient_y)
patient_x = kb.transform(patient_x)
patient_x

array([[44.        , 37.81842105, 21.29411765, ...,  0.        ,
         0.        ,  0.        ],
       [75.        , 37.        , 19.75      , ...,  0.        ,
         0.        ,  0.        ],
       [92.        , 37.0125    , 18.3125    , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [64.        , 37.84772727, 19.26666667, ...,  0.        ,
         0.        ,  0.        ],
       [57.        , 37.65714286, 18.        , ...,  0.        ,
         0.        ,  0.        ],
       [82.        , 37.35714286, 18.52941176, ...,  0.        ,
         0.        ,  0.        ]])

In [48]:
# Seperate training set and testing set

from sklearn import model_selection
train_x, test_x, train_y, test_y = model_selection.train_test_split(patient_x, patient_y, test_size = 0.2)
train_y

313    0
164    0
162    1
292    0
219    0
      ..
306    1
209    0
204    0
283    1
58     0
Name: 6, Length: 256, dtype: int64

In [41]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def average_gradient(model, train_x, train_y):
    gradient = np.zeros(len(model))
    for index in range(len(train_x)):
        print(train_x[index])
        print(train_y[index])
        gradient += np.multiply((sigmoid(model.T.dot(train_x[index])) - train_y[index]), train_x[index])
    return gradient / len(data_set)

def logistic_train(train_x, train_y):
    model = np.ones(len(train_x[0]))
    
    learning_rate = 1
    limit = 100
    for i in range(limit):
        model = model - learning_rate * average_gradient(model, train_x[0], train_y)
        learning_rate *= 0.97 
    return model

def logistic_predict(model, data):
    if sigmoid(model * data) > 0.5:
        return True
    else:
        return False

def logistic_predict_prob(model, data):
    return sigmoid(model * data)

def visulization(model):
    pass

w = logistic_train(train_x, train_y)

89.0
1
36.025
0
16.571428571428573
0
143.16129032258064
0
57.064516129032256


KeyError: 4

In [ ]:
from sklearn import metrics, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score

# Create models
forest = RandomForestClassifier(n_estimators = 100)
extraTrees = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
svc = svm.SVC(probability=True)
voting = VotingClassifier(estimators=[('rf', forest), ('et', extraTrees), ('svm', svc)], voting='soft')

for clf, label in zip([forest, extraTrees, svc, voting], ['Random Forest', 'Extra Trees', 'SVM', 'Voting Classifier']):
    scores = cross_val_score(clf, patient_x, patient_y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    
voting_fit = voting.fit(train_x, train_y)

# Predict
test_y_predicted = voting.predict(test_x)

In [ ]:
# Cal AUC
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

In [ ]:
# f1 score
f1 = metrics.f1_score(test_y, test_y_predicted)
f1

In [ ]:
# Testing
# data_xls = pd.read_excel('Testing data.xlsx', 'Info', index_col=None)
# data_xls.to_csv('TestingInfo.csv', encoding='utf-8',index = False)
# data_xls = pd.read_excel('Testing data.xlsx', 'TPR', index_col=None)
# data_xls.to_csv('TestingTPR.csv', encoding='utf-8', index = False)

# infoSet = pd.read_csv("TestingInfo.csv", header = None, skiprows=[0])
# TPRSet = pd.read_csv("TestingTPR.csv", header = None, skiprows=[0])

# aggregation_functions = {2: 'mean', 3: 'mean', 4: 'mean', 5: 'mean', 6: 'mean'}

# TPRaggreg = TPRSet.groupby(TPRSet[0]).aggregate(aggregation_functions)
# mergeSet = pd.merge(infoSet.iloc[:, [True, True, True, False, False, False]], TPRaggreg.iloc[:, [True, True, True, True, True]], on=0)

# # Extract feature
# bacteria = infoSet.iloc[:, 5]
# bacteria = bacteria.rename("bacteria")
# bacteriaSet, bacteria_feature_not_used = str_trim(bacteria)
# # bacteria_feature_not_used = bacteria_feature_not_used.to_numpy().flatten()

# common = infoSet.iloc[:, 3]
# common = common.rename("common")
# commonSet, commonFeature = str_trim(common)
# commonFeature = commonFeature.to_numpy().flatten()

# # One Hot encoding
# bacteria_df = extract_df(bacteriaSet, bacteria_feature)
# common_df = extract_df(commonSet, commonFeature)

# # Concate
# test_x = pd.concat([mergeSet, bacteria_df, common_df], axis=1)
# test_x = test_x.iloc[:, 1:]

# # Cover select k best mask
# mask = kb.get_support()

# test_x = test_x.iloc[:,mask]

# test_y_predicted = voting.predict(test_x)

# test_y_predicted